In [1]:
from __future__ import absolute_import, division, print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np

In [2]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, 28, 28]), x_test.reshape([-1, 28, 28])
x_train, x_test = x_train / 255., x_test / 255.
x_train, x_test = torch.from_numpy(x_train), torch.from_numpy(x_test), 
y_train, y_test = torch.from_numpy(y_train).type(torch.LongTensor), torch.from_numpy(y_test).type(torch.LongTensor)

D:\Conda\envs\Dang\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [3]:
x_train.shape

torch.Size([60000, 28, 28])

In [4]:
trainloader = []

for (i, j) in zip(x_train, y_train):
    trainloader.append([i, j])

trainloader = torch.utils.data.DataLoader(
    trainloader,
    shuffle = True,
    batch_size = 16
)

testloader = []

for (i, j) in zip(x_test, y_test):
    testloader.append([i, j])

testloader = torch.utils.data.DataLoader(
    testloader,
    shuffle = True,
    batch_size = 16
)

In [6]:
class RNNModel(nn.Module):

    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first = True, nonlinearity = 'relu')

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [7]:
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [10]:
for epoch in range(20):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.180
[2,  2000] loss: 0.166
[3,  2000] loss: 0.142
[4,  2000] loss: 0.131
[5,  2000] loss: 0.120
[6,  2000] loss: 0.116
[7,  2000] loss: 0.102
[8,  2000] loss: 0.101
[9,  2000] loss: 0.097
[10,  2000] loss: 0.092
[11,  2000] loss: 0.087
[12,  2000] loss: 0.088
[13,  2000] loss: 0.092
[14,  2000] loss: 0.077
[15,  2000] loss: 0.081
[16,  2000] loss: 0.070
[17,  2000] loss: 0.072
[18,  2000] loss: 0.071
[19,  2000] loss: 0.065
[20,  2000] loss: 0.072
Finished Training


In [11]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %
